In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
df = pd.read_csv('processed_response.zip')

In [5]:
df.head()

,Polls ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score,time_stamp,tokenized_responses,stemmed_responses,clean_responses,string,LOs/ HCs,College,Course
0,12522,The strengths of Plato's approach is his const...,#objectivemorality,2,1,"['The', 'strengths', 'of', ""Plato's"", 'approac...","['the', 'strength', 'of', 'plato', 'approach',...","['strength', 'plato', 'approach', 'construct',...",strength plato approach construct whole framew...,objmorality,AH,AH111
1,12522,In the breakout we discussed if outside the ca...,#objectivemorality,3,1,"['In', 'the', 'breakout', 'we', 'discussed', '...","['in', 'the', 'breakout', 'we', 'discuss', 'if...","['breakout', 'discuss', 'outsid', 'cave', 'big...",breakout discuss outsid cave bigger cave thus ...,objmorality,AH,AH111
2,12522,Back to cmmon confusion time: the section 'und...,#objectivemorality,2,1,"['Back', 'to', 'cmmon', 'confusion', 'time', '...","['back', 'to', 'cmmon', 'confus', 'time', 'the...","['back', 'cmmon', 'confus', 'time', 'section',...",back cmmon confus time section understand inte...,objmorality,AH,AH111
3,12522,Most difficult weakness is that his position w...,#objectivemorality,2,1,"['Most', 'difficult', 'weakness', 'is', 'that'...","['most', 'difficult', 'weak', 'is', 'that', 'h...","['difficult', 'weak', 'posit', 'understand', '...",difficult weak posit understand testabl like i...,objmorality,AH,AH111
4,12522,I'm still trying to understand the significanc...,#objectivemorality,2,1,"[""I'm"", 'still', 'trying', 'to', 'understand',...","[""i'm"", 'still', 'tri', 'to', 'understand', 't...","[""i'm"", 'still', 'tri', 'understand', 'signifi...",i'm still tri understand signific cave analog ...,objmorality,AH,AH111


In [8]:
#only take when the column isn't NaN 
df = df[df['string'].notna()]

In [9]:
from nltk import word_tokenize

df_string = df.string

tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(df_string)]

In [10]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

In [20]:
tagged_data[:10]

[TaggedDocument(words=['strength', 'plato', 'approach', 'construct', 'whole', 'framework', 'matt', 'answer', 'preparatori', 'poll', 'focus', 'specif', 'moral', 'knowledg', 'instead', 'set', 'form', 'good', 'framework', 'veri', 'hard', 'attack', 'weak', 'abstract'], tags=['0']),
 TaggedDocument(words=['breakout', 'discuss', 'outsid', 'cave', 'bigger', 'cave', 'thus', 'abl', 'illumin', 'howev', 'still', 'stuck', 'know', 'left', 'cave', 'matt', 'said', 'doesnt', 'matter', 'moral', 'knowledg', 'dont', 'need', 'prove', 'pursu'], tags=['1']),
 TaggedDocument(words=['back', 'cmmon', 'confus', 'time', 'section', 'understand', 'intellig', 'tabl', 'suppos', 'equat', 'knowledg', 'knowledg', 'suppos', 'object', 'true', 'differ', 'moral', 'object', 'moral', 'knowledg', 'question', 'rais', 'rosi', 'follow'], tags=['2']),
 TaggedDocument(words=['difficult', 'weak', 'posit', 'understand', 'testabl', 'like', 'incomprehens', 'peopl', 'struggl', 'quit', 'lot', 'decid', 'whether', 'ani', 'way', 'get', 'ca

In [16]:

for epoch in range(max_epochs):
    # print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=10)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


Note: 
- How to tune a model? 16 min for 10 epoch 

- `model.dv.get_vector(key, norm=True)` key is the tagged key number 

- run get vector from 0 - 10, get the list and append it in the dataset 

In [38]:
vectors_list = pd.Series([model.dv.get_vector(i, norm=True) for i in range(len(df))]).T
len(tagged_data), len(vectors_list)

(181793, 181793)

In [43]:
# import numpy as np
# vectors_list = pd.Series([model.dv.get_vector(i, norm=True) for i in range(len(df))]).T

df.reset_index(drop=True, inplace=True)
df['vec'] = vectors_list
df

,Polls ID,Assessment reports Student ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score,time_stamp,tokenized_responses,stemmed_responses,clean_responses,string,LOs/ HCs,College,Course,vec
0,12522.0,41.0,The strengths of Plato's approach is his const...,#objectivemorality,2.0,1,"['The', 'strengths', 'of', ""Plato's"", 'approac...","['the', 'strength', 'of', 'plato', 'approach',...","['strength', 'plato', 'approach', 'construct',...",strength plato approach construct whole framew...,objmorality,AH,AH111,"[0.025362115, -0.148626, -0.00091813056, 0.285..."
1,12522.0,335.0,In the breakout we discussed if outside the ca...,#objectivemorality,3.0,1,"['In', 'the', 'breakout', 'we', 'discussed', '...","['in', 'the', 'breakout', 'we', 'discuss', 'if...","['breakout', 'discuss', 'outsid', 'cave', 'big...",breakout discuss outsid cave bigger cave thus ...,objmorality,AH,AH111,"[0.4415037, 0.012540177, 0.07028076, 0.2250385..."
2,12522.0,318.0,Back to cmmon confusion time: the section 'und...,#objectivemorality,2.0,1,"['Back', 'to', 'cmmon', 'confusion', 'time', '...","['back', 'to', 'cmmon', 'confus', 'time', 'the...","['back', 'cmmon', 'confus', 'time', 'section',...",back cmmon confus time section understand inte...,objmorality,AH,AH111,"[0.11027405, 0.21714334, -0.24093592, -0.03176..."
3,12522.0,315.0,Most difficult weakness is that his position w...,#objectivemorality,2.0,1,"['Most', 'difficult', 'weakness', 'is', 'that'...","['most', 'difficult', 'weak', 'is', 'that', 'h...","['difficult', 'weak', 'posit', 'understand', '...",difficult weak posit understand testabl like i...,objmorality,AH,AH111,"[0.54325324, -0.109847136, 0.109127834, 0.2592..."
4,12522.0,297.0,I'm still trying to understand the significanc...,#objectivemorality,2.0,1,"[""I'm"", 'still', 'trying', 'to', 'understand',...","[""i'm"", 'still', 'tri', 'to', 'understand', 't...","[""i'm"", 'still', 'tri', 'understand', 'signifi...",i'm still tri understand signific cave analog ...,objmorality,AH,AH111,"[0.2842083, -0.36213496, 0.061850082, 0.411065..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181788,320721.0,301.0,Emailed it to you earlier. The explanation in ...,#IsingModel,4.0,15312,"['Emailed', 'it', 'to', 'you', 'earlier', 'The...","['email', 'it', 'to', 'you', 'earlier', 'the',...","['email', 'earlier', 'explan', 'cosh', 'expres...",email earlier explan cosh express larger one t...,isingmodel,NS,NS162,"[-0.0065087997, 0.05535745, -0.22505933, 0.154..."
181789,320721.0,30.0,https://docs.google.com/document/d/JUbBIZWdCAW...,#IsingModel,4.0,15312,"['https', 'docs', 'google', 'com', 'document',...","['https', 'doc', 'googl', 'com', 'document', '...","['https', 'doc', 'googl', 'com', 'document', '...",https doc googl com document jubbizwdcawhf_tnv...,isingmodel,NS,NS162,"[0.2455073, 0.10223179, -0.12469947, 0.2220300..."
181790,320721.0,236.0,https://docs.google.com/document/d/LBhIrfvdqPf...,#IsingModel,4.0,15312,"['https', 'docs', 'google', 'com', 'document',...","['https', 'doc', 'googl', 'com', 'document', '...","['https', 'doc', 'googl', 'com', 'document', '...",https doc googl com document lbhirfvdqpfci khu...,isingmodel,NS,NS162,"[0.053701255, -0.05834162, -0.3643361, 0.02351..."
181791,320721.0,247.0,. The free energy is F=-NkT*ln(*cosh(dj*beta*<...,#IsingModel,4.0,15312,"['The', 'free', 'energy', 'is', 'F', 'NkT', 'l...","['the', 'free', 'energi', 'is', 'f', 'nkt', 'l...","['free', 'energi', 'f', 'nkt', 'ln', 'cosh', '...",free energi f nkt ln cosh dj beta h thus unmag...,isingmodel,NS,NS162,"[-0.109081015, -0.10870954, -0.17266348, 0.281..."
